In [1]:
import os
import random
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from datasets import Dataset, DatasetDict

# ---------------------------
# Paths for Kaggle
# ---------------------------
DATA_CSV = "/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv"
OUTPUT_DIR = "/kaggle/working/outputs_imdb"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------------------
# Random seed
# ---------------------------
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Models to compare
model_names = [
    "bert-base-uncased",
    "roberta-base",
    "microsoft/deberta-base",
    "google/electra-base-discriminator",
    "distilbert-base-uncased",
]

# Hyperparams
subset_train_size = 10000
subset_val_size = 2000
max_length = 256
batch_size = 16
num_train_epochs_small = 2
batch_size_full = 16
num_train_epochs_full = 3

# ---------------------------
# Load dataset
# ---------------------------
df = pd.read_csv(DATA_CSV)
df['label'] = df['sentiment'].map({'negative': 0, 'positive': 1})
df = df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=RANDOM_SEED)
train_df, val_df = train_test_split(train_df, test_size=0.125, stratify=train_df['label'], random_state=RANDOM_SEED)

train_ds = Dataset.from_pandas(train_df[['review', 'label']])
val_ds   = Dataset.from_pandas(val_df[['review', 'label']])
test_ds  = Dataset.from_pandas(test_df[['review', 'label']])
dataset_dict = DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})

# ---------------------------
# Tokenization
# ---------------------------
def get_tokenizer(model_name):
    return AutoTokenizer.from_pretrained(model_name, use_fast=True)

def preprocess_function(examples, tokenizer):
    return tokenizer(examples["review"], padding="max_length", truncation=True, max_length=max_length)

# ---------------------------
# Custom F1 metric
# ---------------------------
def compute_metrics_binary(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    f1 = f1_score(labels, preds, average="binary")
    precision = precision_score(labels, preds, zero_division=0)
    recall = recall_score(labels, preds, zero_division=0)
    return {"f1": f1, "precision": precision, "recall": recall}

# ---------------------------
# Training utility
# ---------------------------
def train_and_eval(model_name, train_dataset, val_dataset, output_dir, small_run=True):
    tokenizer = get_tokenizer(model_name)
    tokenized_train = train_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
    tokenized_val = val_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
    tokenized_train = tokenized_train.remove_columns(["review"])
    tokenized_val = tokenized_val.remove_columns(["review"])
    tokenized_train.set_format("torch")
    tokenized_val.set_format("torch")

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    per_device = batch_size if small_run else batch_size_full
    epochs = num_train_epochs_small if small_run else num_train_epochs_full

    args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        per_device_train_batch_size=per_device,
        per_device_eval_batch_size=per_device,
        num_train_epochs=epochs,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        fp16=torch.cuda.is_available(),
        save_total_limit=1,
        seed=RANDOM_SEED,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_binary,
    )

    trainer.train()
    eval_metrics = trainer.evaluate()
    trainer.save_model(output_dir)
    return eval_metrics, trainer

# ---------------------------
# Stage 1: Compare 5 models on subset
# ---------------------------
small_train = dataset_dict["train"].shuffle(seed=RANDOM_SEED).select(range(min(subset_train_size, len(dataset_dict["train"]))))
small_val = dataset_dict["validation"].shuffle(seed=RANDOM_SEED).select(range(min(subset_val_size, len(dataset_dict["validation"]))))

results = {}
for model_name in model_names:
    print(f"\n=== Training on subset: {model_name} ===")
    outdir = os.path.join(OUTPUT_DIR, "subset_compare", model_name.replace("/", "_"))
    os.makedirs(outdir, exist_ok=True)
    metrics, _ = train_and_eval(model_name, small_train, small_val, outdir, small_run=True)
    results[model_name] = metrics
    print(model_name, metrics)

# Find best model
best_model_name = sorted(results.items(), key=lambda x: x[1]['eval_f1'], reverse=True)[0][0]
print("Best model from subset:", best_model_name)

# ---------------------------
# Stage 2: Fine-tune best model on full train
# ---------------------------
full_outdir = os.path.join(OUTPUT_DIR, "final_best", best_model_name.replace("/", "_"))
full_metrics, full_trainer = train_and_eval(best_model_name, dataset_dict["train"], dataset_dict["validation"], full_outdir, small_run=False)
print("Validation metrics (full training):", full_metrics)

# Test metrics
tokenizer = get_tokenizer(best_model_name)
tokenized_test = dataset_dict["test"].map(lambda x: preprocess_function(x, tokenizer), batched=True).remove_columns(["review"])
tokenized_test.set_format("torch")
test_metrics = full_trainer.evaluate(eval_dataset=tokenized_test)
print("Test set metrics:", test_metrics)

# ---------------------------
# Stage 3: Inference on 10 random test reviews
# ---------------------------
rand_idxs = random.sample(range(len(dataset_dict["test"])), 10)
samples = [dataset_dict["test"][i] for i in rand_idxs]
pipe_tokenized = tokenizer([s["review"] for s in samples], truncation=True, padding=True, max_length=max_length, return_tensors="pt")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(full_outdir).to(device)
inputs = {k: v.to(device) for k, v in pipe_tokenized.items()}
with torch.no_grad():
    logits = model(**inputs).logits
    preds = logits.argmax(dim=-1).cpu().numpy()

print("\n--- 10 random test predictions ---\n")
for i, s in enumerate(samples):
    print(f"Review: {s['review'][:200]}...")
    print(f"True label: {s['label']}, Predicted: {int(preds[i])}")
    print("-" * 50)

2025-08-10 06:04:13.116553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754805853.473722      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754805853.580391      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



=== Training on subset: bert-base-uncased ===


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Validation metrics (full training): {'eval_loss': 0.8806890845298767, 'eval_f1': 0.9412467635929098, 'eval_precision': 0.9373264577548592, 'eval_recall': 0.9452, 'eval_runtime': 44.2543, 'eval_samples_per_second': 112.983, 'eval_steps_per_second': 3.548, 'epoch': 3.0}


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Test set metrics: {'eval_loss': 0.845896303653717, 'eval_f1': 0.9422655783396376, 'eval_precision': 0.937970669837495, 'eval_recall': 0.9466, 'eval_runtime': 88.9323, 'eval_samples_per_second': 112.445, 'eval_steps_per_second': 3.52, 'epoch': 3.0}

--- 10 random test predictions ---

Review: This made-for-TV film is a brilliant one. This is probably the best and favourite role by BAFTA winning John Thaw (Kavanagh Q.C. and Inspector Morse). Tom Oakley (Thaw) widowed man has lived in a vill...
True label: 1, Predicted: 1
--------------------------------------------------
Review: !!! Spoiler alert!!!<br /><br />The point is, though, that I didn't think this film had an ending TO spoil... I only started watching it in the middle, after Matt had gotten into Sarah's body, but the...
True label: 0, Predicted: 1
--------------------------------------------------
Review: First off, let me start with a quote a friend of mine said while watching this movie: "This entire movie had to have been a d